In [27]:
import numpy as np
import pandas
import string
import urllib
from bs4 import BeautifulSoup as bs
import datetime
import dateutil
from pprint import pprint
import os
import re
from datetime import datetime

pandas.set_option('max_colwidth',-1)
print('ok')

ok


In [28]:
# Read dataset
os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read')

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

### SUMMARY STATISTICS FOR FILENAME DATASET
print(len(music_df))
# 2,973 filenames

print('\n')
print(
    music_df
    .groupby('source')
    .size()
    .sort_values(ascending=False)
)
# Sources:
#DimeaDozen      1804
#Etree            564
#LosslessLegs     275
#TradersDen       330

print('\n')
print(
    music_df['filename'].
    groupby(music_df['filename'].
            apply(lambda x: x.lower().split('.')[-1])).
    size().
    sort_values(ascending=False)
)
# File extensions:
#flac    2798
#shn       92
#mp2       48
#mp3       31
#wav        1
#cue        1
#aud        1
#FLAC       1
print('ok')

Datafile read
Columns renamed
2973


source
DimeaDozen      1804
Etree           564 
TradersDen      330 
LosslessLegs    275 
dtype: int64


filename
flac    2799
shn     92  
mp2     48  
mp3     31  
wav     1   
cue     1   
aud     1   
Name: filename, dtype: int64
ok


In [ ]:
'''
STEPS IN FLOWCHART:
A. Does filename start with the artist name? (False -> 0)
B. Does the date come next? (False -> 0)
C. Does an artist name come next? (False -> 0)

D. Does the artist have an official abbrev? (True -> E, False -> G)

E. Is (abbrev used correctly) OR (date correctly formatted)? (True -> F , False -> 1)
F. Is (abbrev used correctly) AND (date correctly formatted)? (True -> H , False -> 2)

G. Is the date correctly formatted? (True -> H, False -> 1)

H. Does the date immediately follow artist name, no space between? (False -> 3)

J. Do they include the mics parameter? (True -> K, False -> L)
K. Do they use the mics parameter correctly? (True -> L, False -> 4)

L. Is taper used correctly? (False -> 4)
M. Is sbesource used correctly? (False -> 4)
N. Is filetype used correctly? (False -> 4)
P. Does filetype accurately describe the type of file? (True -> 5, False -> 4)
'''
# Researching the .cue file that snuck into the dataset...
#adf = inmusic_df
#adf['Sample File Name'] = adf['Sample File Name'].apply(lambda x: x.lower())
#adf[adf['Sample File Name'].apply(lambda x: '.cue' in x)]
print('ok')

In [29]:
# GET LIST OF OFFICIAL ABBREVIATIONS

print('Retrieving from etree')
pageurl = 'http://wiki.etree.org/index.php?page=BandAbbreviations'
page = urllib.request.urlopen(pageurl).read()
soup = bs(page, "html.parser")
tables = soup.find_all('table')

#for thistable in tables:
#    print(thistable)
#    print('---------')

bandtable = tables[1]
bands = bandtable.find_all('tr')
allbands = []

print('Thinking...')
for thisband in bands[1:]:
    theirdata = thisband.find_all('td')
    oneband = {}
    oneband['name'] = theirdata[0].text
    oneband['abbrev'] = theirdata[1].text
    try:
        oneband['otherabbrev'] = theirdata[2].text
    except:
        pass
    try:
        oneband['notes'] = theirdata[3].text
    except:
        pass
    allbands.append(oneband)
abbrev_df = pandas.DataFrame.from_records(allbands)
abbrev_df.index.name = 'rownumber'

column_order = ['name', 'abbrev', 'otherabbrev', 'notes']
abbrev_df = abbrev_df[column_order]
abbrev_df = abbrev_df.fillna('')

# Trim one space from beginning and end of band name
abbrev_df['name'] = abbrev_df['name'].apply(lambda x: str(x).strip())

# Trim away brackets from beginning and end of band name (for some bands)
abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[0])) == '['), 
                             abbrev_df['name'].apply(lambda x: str(x)[1:]), 
                             abbrev_df['name']) 

abbrev_df['name'] = abbrev_df['name'] = np.where((abbrev_df['name'].apply(lambda x: (str(x)[len(str(x))-1])) == ']'),
                              abbrev_df['name'].apply(lambda x: str(x)[:-1]), 
                              abbrev_df['name'])

abbrev_df['abbrev'] = abbrev_df['abbrev'].apply(lambda x: str(x).lower().strip())
abbrev_df['otherabbrev'] = abbrev_df['otherabbrev'].apply(lambda x: str(x).lower().strip())

#abbrev_df['abbrev'].apply(lambda x: '|'+str(x)+'|')

abbrev_df.to_csv('abbreviations.csv')
os.listdir()
#print('ok')
#pprint(abbrev_df.values.tolist())
canonical_abbrevs = abbrev_df['abbrev'][abbrev_df['abbrev'] != ''].drop_duplicates().tolist()
#canonical_abbrevs

# Get variant abbreviations
variant_abbrevs = []
for x in abbrev_df['otherabbrev'].drop_duplicates():
    if (',' in x):
        for y in x.split(','):
            variant_abbrevs.append(y.strip())
    else:
        if (x.strip() != ''):
            variant_abbrevs.append(x.strip())

variant_abbrevs = sorted(variant_abbrevs)

# Fix wilco by moving it from variant to canonical
canonical_abbrevs.append('wilco')
variant_abbrevs.remove('wilco')

# Strip and lower
canonical_abbrevs = [x.lower().strip() for x in canonical_abbrevs]    
variant_abbrevs = [x.lower().strip() for x in variant_abbrevs]    

print('Parsed {0:,.0f} canonical abbreviations and {1:,.0f} variant abbreviations.'.format(len(canonical_abbrevs), len(variant_abbrevs)))

Retrieving from etree
Thinking...
Parsed 156 canonical abbreviations and 61 variant abbreviations.


In [30]:
debug = 1
markers = re.compile('[\.\-\_\ ]*')
endmarkers = re.compile('[\.\-\_\ ]$')
startswithbadness = '^[\s\_\-]*(dis|cd|track|artist|unknown)[\s\_\-]*'
startswithdigit = re.compile('^[\d]+')
dateformat = re.compile('[1-9]\d*[\s\/\-]*\d+[\s\/\-]*\d+')
daterightafterband = re.compile('[\d\s\_\-\.\ ]$')
bandtrimmer = re.compile('\s*[\s\_\-\.\ ]$')
scbands = re.compile('(calloway|earth|gregory|bley|smith|jimmy|zeppelin|redford|lizzy|beck|dust|peter gabriel|stones|peppers|sinatra)')
ends_with_dash_or_whatever = re.compile('\s*[\-\_\.]$')

os.chdir('/home/idies/workspace/raddick_folksonomy/data/')
os.getcwd()
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df
# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})

print('Filenames in dataset: {0:,.0f}'.format(len(music_df)))

# First, strip filename and convert to lowercase.
music_df['filename'] = music_df['filename'].apply(lambda x: x.lower().strip())

# Next, get filetype (whatever is after the final period)
music_df['filetype'] = music_df['filename'].apply(lambda x: x.split('.')[-1].strip())
music_df['remainder'] = music_df['filename'].apply(lambda x: x[0:x.rfind('.')])


zeros_df = pandas.DataFrame(data=None, columns=music_df.columns)
#ones_df = pandas.DataFrame(data=None, columns=music_df.columns)
#twos_df = pandas.DataFrame(data=None, columns=music_df.columns)
#threes_df = pandas.DataFrame(data=None, columns=music_df.columns)
#fours_df = pandas.DataFrame(data=None, columns=music_df.columns)

zeros_df.index.name = music_df.index.name
#ones_df.index.name = music_df.index.name
#twos_df.index.name = music_df.index.name
#threes_df.index.name = music_df.index.name
#fours_df.index.name = music_df.index.name

# If there are no numbers in the remainder part of filename, score as zero and remove
print('\nStep 1. Filenames with no numbers in remainder part: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search('\d', x) == None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2a. Filenames with track/cd/disc followed by number with optional whitespace: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search(startswithbadness, x) != None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2b. Starts with a number but has no dates and is not a special case band: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['filename'].apply(lambda x: 
               (re.search(startswithdigit, x) != None) & 
               (re.search(dateformat, x) == None) &
               (re.search(scbands, x) == None)                                                
              )])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

# If starts with track or disc or cd and has a number immediately after, score as zero.
print('\nStep 2c. Remainder has only number: score = 0')
zeros_df = zeros_df.append(music_df.loc[music_df['remainder'].apply(lambda x: re.search('\D', x) == None)])
music_df = music_df[~music_df.isin(zeros_df)].dropna()
print('Filenames remaining in dataset: {0:,.0f}'.format(len(music_df)))

Filenames in dataset: 2,973

Step 1. Filenames with no numbers in remainder part: score = 0
Filenames remaining in dataset: 2,955

Step 2a. Filenames with track/cd/disc followed by number with optional whitespace: score = 0
Filenames remaining in dataset: 2,735

Step 2b. Starts with a number but has no dates and is not a special case band: score = 0
Filenames remaining in dataset: 1,805

Step 2c. Remainder has only number: score = 0
Filenames remaining in dataset: 1,803


In [31]:
# Parse up to first digit as band name
music_df['band'] = music_df['remainder'].apply(lambda x: x[0:re.search('\d', x).start()])

# Find whether the date immediately follows the band name (as the format demands)
music_df['date_right_after_band'] = False
music_df.loc[music_df['band'].apply(lambda x: re.search(daterightafterband, x) == None), 'date_right_after_band'] = True

# Strip and lower band names
music_df['band'] = music_df['band'].apply(lambda x: x.strip().lower())

# If the band name has a _ or - immediately after, strip that away
music_df.loc[music_df['band'].apply(lambda x: re.search(ends_with_dash_or_whatever, x) != None), 
             'band'] = music_df['band'][music_df['band'].apply(lambda x: re.search(ends_with_dash_or_whatever, x) != None)
                                       ].apply(lambda x: x[0:re.search(ends_with_dash_or_whatever, x).start()])

# Fix mistaken bands d, t, a, n, and tt
music_df.loc[
    (music_df['band'] == 'd') | (music_df['band'] == 't') | (music_df['band'] == 'a') | (music_df['band'] == 'n') | (music_df['band'] == 'tt'), 
    'date_right_after_band'] = False
music_df.loc[
    (music_df['band'] == 'd') | (music_df['band'] == 't') | (music_df['band'] == 'a') | (music_df['band'] == 'n') | (music_df['band'] == 'tt'), 
    'band'] = ''

# Fix U2
music_df.loc[music_df['band'] == 'u', 'band'] = 'u2'

# A few other bands to fix
music_df.loc[(music_df['band'] == 'radiohead_lollapalooza'), 'band'] = 'radiohead'


# Some specific rows to fix
#143
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'band'] = 'neil young'
music_df.loc[music_df['id'] == 'f808ec4b46eb140fca6b9b73f6cbff76ee0e6641', 'date_right_after_band'] = False
#185
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'date_right_after_band'] = False
#192
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'band'] = 'the ramones'
music_df.loc[music_df['id'] == 'ff3fe93793664814e48139fbd51022e73347ec86', 'date_right_after_band'] = False
#234
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'band'] = 'rush'
music_df.loc[music_df['id'] == 'ebb2e1757bdb2a01716acefca2697fd718d9a8c1', 'date_right_after_band'] = False
#244
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'band'] = 'deadco'
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 'date_right_after_band'] = False
#781
music_df.loc[music_df['id'] == '587888.html', 'band'] = 'phish-sleeptrain'
music_df.loc[music_df['id'] == '587888.html', 'date_right_after_band'] = False
#840
music_df.loc[music_df['id'] == '565503.html', 'band'] = 'the cure'
music_df.loc[music_df['id'] == '565503.html', 'date_right_after_band'] = False
#945
music_df.loc[music_df['id'] == '565650.html', 'band'] = 'maku'
music_df.loc[music_df['id'] == '565650.html', 'date_right_after_band'] = False
#1046
music_df.loc[music_df['id'] == '565793.html', 'band'] = 'gordon gatton'
music_df.loc[music_df['id'] == '565793.html', 'date_right_after_band'] = False
#1087
music_df.loc[music_df['id'] == '565852.html', 'band'] = 'the obsessed'
music_df.loc[music_df['id'] == '565852.html', 'date_right_after_band'] = False
#1115
music_df.loc[music_df['id'] == '565891.html', 'band'] = 'nickcave'
#1139
music_df.loc[music_df['id'] == '565923.html', 'band'] = 'vm'
music_df.loc[music_df['id'] == '565923.html', 'date_right_after_band'] = False
#1149
music_df.loc[music_df['id'] == '565934.html', 'band'] = 'l-ficher-g-baton'
music_df.loc[music_df['id'] == '565934.html', 'date_right_after_band'] = False
#1162
music_df.loc[music_df['id'] == '565954.html', 'band'] = 'ehg'
music_df.loc[music_df['id'] == '565954.html', 'date_right_after_band'] = False
#1233
music_df.loc[music_df['id'] == '566050.html', 'band'] = 'black sabbath'
#1245
music_df.loc[music_df['id'] == '566066.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566066.html', 'date_right_after_band'] = False
#1277
music_df.loc[music_df['id'] == '566114.html', 'band'] = 'mavis staples'
#1288
music_df.loc[music_df['id'] == '566127.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566127.html', 'date_right_after_band'] = False
#1295
music_df.loc[music_df['id'] == '566136.html', 'band'] = 'karma to burn'
music_df.loc[music_df['id'] == '566136.html', 'date_right_after_band'] = False
#1301
music_df.loc[music_df['id'] == '566144.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566144.html', 'date_right_after_band'] = False
#1312
music_df.loc[music_df['id'] == '566155.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566155.html', 'date_right_after_band'] = False
#1314
music_df.loc[music_df['id'] == '566157.html', 'band'] = 'the jam'
music_df.loc[music_df['id'] == '566157.html', 'date_right_after_band'] = False
#1355
music_df.loc[music_df['id'] == '566208.html', 'band'] = 'danish radio big band'
music_df.loc[music_df['id'] == '566208.html', 'date_right_after_band'] = False
#1377
music_df.loc[music_df['id'] == '566234.html', 'band'] = 'jam'
#1393
music_df.loc[music_df['id'] == '566252.html', 'band'] = 'casting crowns'
#1403
music_df.loc[music_df['id'] == '566264.html', 'band'] = 'paul mccartney'
music_df.loc[music_df['id'] == '566264.html', 'date_right_after_band'] = False
#1414
music_df.loc[music_df['id'] == '566278.html', 'band'] = 'dizzy mizz lizzy'
music_df.loc[music_df['id'] == '566278.html', 'date_right_after_band'] = False
#1533
music_df.loc[music_df['id'] == '566439.html', 'band'] = 'hw'
music_df.loc[music_df['id'] == '566439.html', 'date_right_after_band'] = False
#1596
music_df.loc[music_df['id'] == '566526.html', 'band'] = 'jam'
music_df.loc[music_df['id'] == '566526.html', 'date_right_after_band'] = False
#1683
music_df.loc[music_df['id'] == '566657.html', 'band'] = 'van morrison'
music_df.loc[music_df['id'] == '566657.html', 'date_right_after_band'] = False
#1706
music_df.loc[music_df['id'] == '566685.html', 'band'] = 'glen hansard'
music_df.loc[music_df['id'] == '566685.html', 'date_right_after_band'] = False
#1737
music_df.loc[music_df['id'] == '566734.html', 'band'] = 'peter &amp; the test tube babies'
music_df.loc[music_df['id'] == '566734.html', 'date_right_after_band'] = False
#1759
music_df.loc[music_df['id'] == '566756.html', 'band'] = 'marillion'
music_df.loc[music_df['id'] == '566756.html', 'date_right_after_band'] = False
#1784
music_df.loc[music_df['id'] == '566783.html', 'band'] = 'demiricous'
music_df.loc[music_df['id'] == '566783.html', 'date_right_after_band'] = False
#1790
music_df.loc[music_df['id'] == '566789.html', 'band'] = 'the clash'
music_df.loc[music_df['id'] == '566789.html', 'date_right_after_band'] = False
#1796
music_df.loc[music_df['id'] == '566798.html', 'band'] = 'keith jarrett'
music_df.loc[music_df['id'] == '565954.html', 'date_right_after_band'] = False
#1808
music_df.loc[music_df['id'] == '566810.html', 'band'] = 'lucie vitkova-ingelfritz'
music_df.loc[music_df['id'] == '566810.html', 'date_right_after_band'] = False
#1811
music_df.loc[music_df['id'] == '566814.html', 'band'] = 'mjarre'
#1869
music_df.loc[music_df['id'] == '566896.html', 'mics_etc'] = 'macca'
#1875  
music_df.loc[music_df['id'] == '566903.html', 'band'] = 'dixie chicks'
music_df.loc[music_df['id'] == '566903.html', 'date_right_after_band'] = False
#1933
music_df.loc[music_df['id'] == '566973.html', 'band'] = 'chess smith trio'
music_df.loc[music_df['id'] == '566973.html', 'date_right_after_band'] = False
#1984
music_df.loc[music_df['id'] == '567047.html', 'band'] = 'charlie cunningham'
music_df.loc[music_df['id'] == '567047.html', 'date_right_after_band'] = False
#2040
music_df.loc[music_df['id'] == '567120.html', 'band'] = 'neil young'
music_df.loc[music_df['id'] == '567120.html', 'date_right_after_band'] = False
#2270
music_df.loc[music_df['id'] == '567402.html', 'band'] = 'nickcave'
music_df.loc[music_df['id'] == '567402.html', 'date_right_after_band'] = False
#2053
music_df.loc[music_df['id'] == '567140.html', 'band'] = 'kiss'
music_df.loc[music_df['id'] == '567140.html', 'date_right_after_band'] = False
#2092
music_df.loc[music_df['id'] == '567194.html', 'band'] = 'acdc'
music_df.loc[music_df['id'] == '567194.html', 'date_right_after_band'] = False
#2099
music_df.loc[music_df['id'] == '567202.html', 'band'] = 'lk'
#2200
music_df.loc[music_df['id'] == '567318.html', 'band'] = 'vm'
music_df.loc[music_df['id'] == '567318.html', 'date_right_after_band'] = False
#2270
music_df.loc[music_df['id'] == '567402.html', 'band'] = 'nickcave'
music_df.loc[music_df['id'] == '567402.html', 'date_right_after_band'] = False
#2374
music_df.loc[music_df['id'] == '567536.html', 'band'] = ''
music_df.loc[music_df['id'] == '565650.html', 'date_right_after_band'] = False
#2516 
music_df.loc[music_df['id'] == '567720.html', 'band'] = 'david bowie'
music_df.loc[music_df['id'] == '567720.html', 'date_right_after_band'] = False
#2558
music_df.loc[music_df['id'] == '567771.html', 'band'] = 'daywave'
music_df.loc[music_df['id'] == '567771.html', 'date_right_after_band'] = False
#2569
music_df.loc[music_df['id'] == '567795.html', 'band'] = 'rob brown quartet'
music_df.loc[music_df['id'] == '567795.html', 'date_right_after_band'] = False
#2634
music_df.loc[music_df['id'] == '567879.html', 'band'] = 'modern baseball'
music_df.loc[music_df['id'] == '567879.html', 'date_right_after_band'] = False
#2823
music_df.loc[music_df['id'] == '127023.html', 'band'] = 'neil young'
#2828
music_df.loc[music_df['id'] == '127033.html', 'band'] = 'sm pf'
music_df.loc[music_df['id'] == '127033.html', 'date_right_after_band'] = False

## SECOND ROUND found by Jordan 3/27/2018
#186
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'band'] = ''
music_df.loc[music_df['id'] == '5a3b55bc30d8dabb1023d442759c34dd70edebb1', 'date_right_after_band'] = False
#636
music_df.loc[music_df['id'] == '587838.html', 'band'] = ''
music_df.loc[music_df['id'] == '587838.html', 'date_right_after_band'] = False
#778
music_df.loc[music_df['id'] == '565641.html', 'band'] = ''
music_df.loc[music_df['id'] == '565641.html', 'date_right_after_band'] = False
#950
music_df.loc[music_df['id'] == '566104.html', 'band'] = 'bob dylan'
music_df.loc[music_df['id'] == '566104.html', 'date_right_after_band'] = False
#1002
music_df.loc[music_df['id'] == '566232.html', 'band'] = ''
music_df.loc[music_df['id'] == '566232.html', 'date_right_after_band'] = False
#1793
music_df.loc[music_df['id'] == '127205.html', 'band'] = ''
music_df.loc[music_df['id'] == '127205.html', 'date_right_after_band'] = False
#881
music_df.loc[music_df['id'] == '565901.html', 'band'] = 'whitford'
music_df.loc[music_df['id'] == '565901.html', 'date_right_after_band'] = False
#1181
music_df.loc[music_df['id'] == '566727.html', 'band'] = 'buben'
music_df.loc[music_df['id'] == '566727.html', 'date_right_after_band'] = False
#1374
music_df.loc[music_df['id'] == '567234.html', 'band'] = 'gnr'
music_df.loc[music_df['id'] == '567234.html', 'date_right_after_band'] = False
#1587
music_df.loc[music_df['id'] == '567731.html', 'band'] = 'the brew'
music_df.loc[music_df['id'] == '567731.html', 'date_right_after_band'] = False

#music_df[['id', 'filename', 'band', 'date_right_after_band']].head(3)
print('Parsed band abbrevs for {0:,.0f} filenames (out of {1:,.0f} total).'.format(len(music_df[music_df['band'] != '']), len(music_df)))
print('Found {0:,.0f} distinct band abbrevs.'.format(len(music_df['band'][music_df['band'] != ''].drop_duplicates())))

Parsed band abbrevs for 1,543 filenames (out of 1,803 total).
Found 758 distinct band abbrevs.


In [32]:
# Check band names vs. list of canonical abbreviations
music_df['abbrev_type'] = 'xxx'
music_df.loc[music_df['band'].isin(canonical_abbrevs), 'abbrev_type'] = 'canonical'
# Check band names vs. list of variant abbreviations
music_df.loc[music_df['band'].isin(variant_abbrevs), 'abbrev_type'] = 'variant'
# No band name
music_df.loc[music_df['band'] == '', 'abbrev_type'] = 'blank'

# Band names used incorrect abbreviations
incorrect = ['phish', 'widespreadpanic', 'jg+ms', 'deadco', 'deadandco', 'dc', 'jgms']
incorrect += ['dead and company', 'dead.and.co', 'dead.and.co', 'dac', 'dandc']
incorrect += ['deadco-', 'dead&co', 'dead&amp;co', 'dead&amp;company', 'deadandcompany']
incorrect += ['deadco.irvine.', 'deadfeat', 'gov`t mule']
music_df.loc[music_df['band'].isin(incorrect), 'abbrev_type'] = 'incorrect'

# Band names not in abbreviation list
notfound = ['loslobos', 'deanmonkey&amp;thedropouts', 'ttb', 'crb', 'deadandcompany', 'ny', 'gsbg', 'bd']
notfound += ['jefferson starship', 'carlton, larry -', 'the who', 's marriott', 'unknown artist']
notfound += ['tal wilkenfeld', 'ph&amp;tl', 'sb', 'radiohead', 'theclash_', 'primus']

vc = music_df['band'][music_df['band'] != ''].value_counts()
# If a band name appears 4 or fewer times, assume it's a misc band not found in the list.
for x in (vc[(vc<5)].index.sort_values().tolist()):
    if ((x not in canonical_abbrevs) and (x not in variant_abbrevs) and (x not in notfound) and (x not in incorrect)):
        notfound.append(x)
        
notfound += ['u2', 'the jam', 'theclash', 'carlton, larry']
music_df.loc[music_df['band'].isin(notfound), 'abbrev_type'] = 'notfound'

#music_df[['id', 'filename', 'band', 'date_right_after_band', 'abbrev_type']][music_df['abbrev_type'] == 'xxx'].sample(10)
music_df.groupby('abbrev_type').size()

abbrev_type
blank        260 
canonical    329 
incorrect    107 
notfound     1074
variant      33  
dtype: int64

In [33]:
# Get Jeremy's list of bands
bands_df = pandas.read_csv('bands_verified.csv', index_col=0, low_memory=False)#, dtype={'band': 'S1024', 'band_or_not': 'S6', 'canonical_abbreviation': 'S16', 'notes': 'S4096'})

bands_df.index = pandas.to_numeric(bands_df.index)

for thiscol in ['band', 'canonical_abbreviation', 'non_canonical', 'notes']:
    bands_df[thiscol] = bands_df[thiscol].apply(lambda x: str(x)[1:-1])

bands_df['band_or_not'] = bands_df['band_or_not'].apply(lambda x: x.lower())


music_df = music_df.merge(bands_df, how='left', on='band')
#music_df[music_df['band_or_not'] == 'yes'].sample(10)

music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'yes'), 'abbrev_type'] = 'jeremy'
music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'maybe'), 'abbrev_type'] = 'notband'
music_df.loc[(music_df['abbrev_type'] == 'notfound') & (music_df['band_or_not'] == 'no'), 'abbrev_type'] = 'notband'

music_df = music_df.drop(['band_or_not', 'canonical_abbreviation', 'non_canonical'], axis=1)
music_df = music_df.rename(columns={'notes': 'jeremy_band_finding_notes'})
music_df['jeremy_band_finding_notes'] = music_df['jeremy_band_finding_notes'].fillna('')

music_df.groupby('abbrev_type').size().sort_values(ascending=False)

abbrev_type
jeremy       896
canonical    329
blank        260
notband      178
incorrect    107
variant      33 
dtype: int64

In [34]:
date1 = re.compile('\d+[\-\/\ —]+\d+[\-\/\ —]+\d+')
date2 = re.compile('(\d{8})')
date3 = re.compile('(19|20)\d{2}[-]\D+[-]\d{1,2}')

music_df['parsed_date'] = pandas.to_datetime(np.nan)
music_df['mics_etc'] = ''
music_df['date_format'] = ''

# Parse everything after the first digit for analysis of dates
music_df['after_first_digit'] = music_df['remainder'].apply(lambda x: x[re.search('\d', x).start():len(x)])

# Fix for u2 since they have a number as their band name
music_df.loc[music_df['band'] == 'u2', 
            'after_first_digit'] = music_df['after_first_digit'][
    music_df['band'] == 'u2'].apply(
lambda x: x[re.search('^2[\s\_\-]*', x).end():len(x)])

# Parse date as yyyy-mm-dd
music_df['full_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:10]), format='%Y-%m-%d', errors='coerce').dropna()
music_df['full_date_mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[10:])
music_df.loc[(music_df['date_format'] == '') & (music_df['full_date'].notnull()), 'date_format'] = 'yyyy-mm-dd'
music_df.loc[music_df['date_format'] == 'yyyy-mm-dd', 'parsed_date'] = music_df['full_date'][music_df['date_format'] == 'yyyy-mm-dd']
music_df.loc[music_df['date_format'] == 'yyyy-mm-dd', 'mics_etc'] = music_df['full_date_mics_etc'][music_df['date_format'] == 'yyyy-mm-dd']

# Parse date as yy-mm-dd
music_df['yy_date'] = pandas.to_datetime(music_df['after_first_digit'].apply(lambda x: x[0:8]), format='%y-%m-%d', errors='coerce').dropna()
music_df['yy_date_mics_etc'] = music_df['after_first_digit'].apply(lambda x: x[8:])
music_df.loc[(music_df['date_format'] == '') & (music_df['yy_date'].notnull()), 'date_format'] = 'yy-mm-dd'
music_df.loc[music_df['date_format'] == 'yy-mm-dd', 'parsed_date'] = music_df['yy_date'][music_df['date_format'] == 'yy-mm-dd']
music_df.loc[music_df['date_format'] == 'yy-mm-dd', 'mics_etc'] = music_df['yy_date_mics_etc'][music_df['date_format'] == 'yy-mm-dd']

# Parse date as date1 format
music_df['date1_date'] = pandas.to_datetime(music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date1, x)))
                             ].apply(lambda x: x[re.search(date1, x).start():re.search(date1, x).end()]), errors='coerce')
music_df['date1_date_mics_etc'] = music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date1, x)))
                             ].apply(lambda x: x[re.search(date1, x).end():])
music_df.loc[(music_df['date_format'] == '') & (music_df['date1_date'].notnull()), 'date_format'] = 'date1'
music_df.loc[music_df['date_format'] == 'date1', 'parsed_date'] = music_df['date1_date']
music_df.loc[music_df['date_format'] == 'date1', 'mics_etc'] = music_df['date1_date_mics_etc']

# Parse date as date2 format
music_df['date2_date'] = pandas.to_datetime(music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date2, x)))
                             ].apply(lambda x: x[re.search(date2, x).start():re.search(date2, x).end()]), errors='coerce')
music_df['date2_date_mics_etc'] = music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date2, x)))
                             ].apply(lambda x: x[re.search(date2, x).end():])
music_df.loc[(music_df['date_format'] == '') & (music_df['date2_date'].notnull()), 'date_format'] = 'date2'
music_df.loc[music_df['date_format'] == 'date2', 'parsed_date'] = music_df['date2_date']
music_df.loc[music_df['date_format'] == 'date2', 'mics_etc'] = music_df['date2_date_mics_etc']

# Parse date as date3 format
music_df['date3_date'] = pandas.to_datetime(music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date3, x)))
                             ].apply(lambda x: x[re.search(date3, x).start():re.search(date3, x).end()]), errors='coerce')
music_df['date3_date_mics_etc'] = music_df['after_first_digit'][(music_df['date_format'] == '') & 
                             (music_df['after_first_digit'].apply(lambda x: re.search(date3, x)))
                             ].apply(lambda x: x[re.search(date3, x).end():])
music_df.loc[(music_df['date_format'] == '') & (music_df['date3_date'].notnull()), 'date_format'] = 'date3'
music_df.loc[music_df['date_format'] == 'date3', 'parsed_date'] = music_df['date3_date']
music_df.loc[music_df['date_format'] == 'date3', 'mics_etc'] = music_df['date3_date_mics_etc']


# Remove interim columns
music_df = music_df.drop(
    ['full_date', 'full_date_mics_etc', 'yy_date', 'yy_date_mics_etc', 
     'date1_date', 'date1_date_mics_etc', 'date2_date', 'date2_date_mics_etc',
     'date3_date', 'date3_date_mics_etc'], axis=1)


#
#music_df[['filename', 'after_first_digit', 'parsed_date', 'mics_etc', 'date_format']][music_df['date_format'] == '']
                                                         
#music_df[['filename', 'after_first_digit', 'parsed_date', 'mics_etc', 'date_format']
#        ]
music_df.groupby('date_format').size()

date_format
              340 
date1         69  
date2         96  
date3         5   
yy-mm-dd      123 
yyyy-mm-dd    1170
dtype: int64

In [35]:
music_df['date_by_hand'] = pandas.to_datetime(np.nan)

#82
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 
             'date_by_hand'] = pandas.to_datetime('1997-06-22')
music_df.loc[music_df['id'] == '3c39e32bf0918ec3190ecda08442eba0d8b2faa8', 
             'date_by_hand_mics_etc'] = 'beausolield1tr01'
#191
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 
             'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == 'c2b7c10aa42aa6f55c8537ece72ed5726ae6b1b3', 
             'date_by_hand_mics_etc'] = '.01'
#218
music_df.loc[music_df['id'] == '587337.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-03')
music_df.loc[music_df['id'] == '587337.html', 
             'date_by_hand_mics_etc'] = '01'
#219
music_df.loc[music_df['id'] == '587339.html', 
             'date_by_hand'] = pandas.to_datetime('2016-02-19')
#316
music_df.loc[music_df['id'] == '587454.html', 
             'date_by_hand'] = pandas.to_datetime('2016-01-30')
music_df.loc[music_df['id'] == '587454.html', 
             'date_by_hand_mics_etc'] = '01'
#409
music_df.loc[music_df['id'] == '587572.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '587572.html', 
             'date_by_hand_mics_etc'] = '01'
#421
music_df.loc[music_df['id'] == '587588.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-10')
music_df.loc[music_df['id'] == '587588.html', 
             'date_by_hand_mics_etc'] = '01'
#438
music_df.loc[music_df['id'] == '587606.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '587606.html', 
             'date_by_hand_mics_etc'] = '_t01'
#467
music_df.loc[music_df['id'] == '587640.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '587640.html', 
             'date_by_hand_mics_etc'] = '01'
#588
music_df.loc[music_df['id'] == '587786.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '587786.html', 
             'date_by_hand_mics_etc'] = ' george,wa t15 crosseyed and painless'
#676
music_df.loc[music_df['id'] == '587888.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '587888.html', 
             'date_by_hand_mics_etc'] = '.01'
#749
music_df.loc[music_df['id'] == '565561.html', 
             'date_by_hand'] = pandas.to_datetime('2011-09-06')
#786
music_df.loc[music_df['id'] == '565664.html', 
             'date_by_hand'] = pandas.to_datetime('2016-05-31')
music_df.loc[music_df['id'] == '565664.html', 
             'date_by_hand_mics_etc'] = ' 210'
#824
music_df.loc[music_df['id'] == '565762.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-03')
#833
music_df.loc[music_df['id'] == '565781.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-03')
#837
music_df.loc[music_df['id'] == '565793.html', 
             'date_by_hand'] = pandas.to_datetime('1981-06-27')
music_df.loc[music_df['id'] == '565793.html', 
             'date_by_hand_mics_etc'] = '01'
#876
music_df.loc[music_df['id'] == '565891.html', 
             'date_by_hand'] = pandas.to_datetime('1989-02-18')
music_df.loc[music_df['id'] == '565891.html', 
             'date_by_hand_mics_etc'] = '_d6_909_t01'

#891
music_df.loc[music_df['id'] == '565923.html', 
             'date_by_hand'] = pandas.to_datetime('1997-07-05')
music_df.loc[music_df['id'] == '565923.html', 
             'date_by_hand_mics_etc'] = '_disc 02 track 02'
#903
music_df.loc[music_df['id'] == '565954.html', 
             'date_by_hand'] = pandas.to_datetime('2016-04-29')
music_df.loc[music_df['id'] == '565954.html', 
             'date_by_hand_mics_etc'] = '- t01'
#909
music_df.loc[music_df['id'] == '565970.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-06')
music_df.loc[music_df['id'] == '565970.html', 
             'date_by_hand_mics_etc'] = ' destinies entwined'
#937
music_df.loc[music_df['id'] == '566066.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-07')
music_df.loc[music_df['id'] == '566066.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#950
music_df.loc[music_df['id'] == '566104.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566104.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#954
music_df.loc[music_df['id'] == '566114.html', 
             'date_by_hand'] = pandas.to_datetime('2016-06-25')
music_df.loc[music_df['id'] == '566114.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#957
music_df.loc[music_df['id'] == '566127.html', 
             'date_by_hand'] = pandas.to_datetime('1980-03-07')
music_df.loc[music_df['id'] == '566127.html', 
             'date_by_hand_mics_etc'] = ' 16'
#960
music_df.loc[music_df['id'] == '566132.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-06')
music_df.loc[music_df['id'] == '566132.html.html', 
             'date_by_hand_mics_etc'] = '201'
#967
music_df.loc[music_df['id'] == '566148.html', 
             'date_by_hand'] = pandas.to_datetime('2012-11-06')
music_df.loc[music_df['id'] == '566148.html', 
             'date_by_hand_mics_etc'] = '_berlin_16_tr01'
#971
music_df.loc[music_df['id'] == '566157.html', 
             'date_by_hand'] = pandas.to_datetime('1979-11-24')
music_df.loc[music_df['id'] == '566157.html', 
             'date_by_hand_mics_etc'] = '_tr15'
#991
music_df.loc[music_df['id'] == '566210.html', 
             'date_by_hand'] = pandas.to_datetime('2016-01-08')
#996
music_df.loc[music_df['id'] == '566219.html', 
             'date_by_hand'] = pandas.to_datetime('1990-02-17')
music_df.loc[music_df['id'] == '566219.html', 
             'date_by_hand_mics_etc'] = '_d6_909_t01'
#999
music_df.loc[music_df['id'] == '566223.html', 
             'date_by_hand'] = pandas.to_datetime('1988-07-09')
music_df.loc[music_df['id'] == '566223.html', 
             'date_by_hand_mics_etc'] = '_07'
#1004
music_df.loc[music_df['id'] == '566234.html', 
             'date_by_hand'] = pandas.to_datetime('1979-11-18')
music_df.loc[music_df['id'] == '566234.html', 
             'date_by_hand_mics_etc'] = '_15'
#1014
music_df.loc[music_df['id'] == '566252.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-09')
music_df.loc[music_df['id'] == '566252.html', 
             'date_by_hand_mics_etc'] = '01'
#1020
music_df.loc[music_df['id'] == '566264.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-10')
music_df.loc[music_df['id'] == '566264.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#1040
music_df.loc[music_df['id'] == '566331.html', 
             'date_by_hand'] = pandas.to_datetime('2012-11-06')
music_df.loc[music_df['id'] == '566331.html', 
             'date_by_hand_mics_etc'] = '_berlin_16_tr01'
#1154
music_df.loc[music_df['id'] == '566657.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-07')
music_df.loc[music_df['id'] == '566657.html', 
             'date_by_hand_mics_etc'] = ' (biddie)_track 17'
#1166
music_df.loc[music_df['id'] == '566680.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-11')
music_df.loc[music_df['id'] == '566680.html', 
             'date_by_hand_mics_etc'] = ' 115'
#1209
music_df.loc[music_df['id'] == '566783.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-15')
music_df.loc[music_df['id'] == '566783.html', 
             'date_by_hand_mics_etc'] = ' - t01'
#1297
music_df.loc[music_df['id'] == '567003.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-17')
music_df.loc[music_df['id'] == '567003.html', 
             'date_by_hand_mics_etc'] = '_1851'
#1309
music_df.loc[music_df['id'] == '567053.html', 
             'date_by_hand'] = pandas.to_datetime('1981-08-20')
music_df.loc[music_df['id'] == '567053.html', 
             'date_by_hand_mics_etc'] = 'scruzcivic'
#1335
music_df.loc[music_df['id'] == '567140.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-20')
#1339
music_df.loc[music_df['id'] == '567146.html', 
             'date_by_hand'] = pandas.to_datetime('2013-04-01')
#1366
music_df.loc[music_df['id'] == '567220.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-02')
music_df.loc[music_df['id'] == '567220.html', 
             'date_by_hand_mics_etc'] = ')'
#1421
music_df.loc[music_df['id'] == '567318.html', 
             'date_by_hand'] = pandas.to_datetime('2011-09-02')
music_df.loc[music_df['id'] == '567318.html', 
             'date_by_hand_mics_etc'] = '_disc 01 track 04'
#1445
music_df.loc[music_df['id'] == '567376.html', 
             'date_by_hand'] = pandas.to_datetime('1975-08-17')
music_df.loc[music_df['id'] == '567376.html', 
             'date_by_hand_mics_etc'] = '_04'
#1468
music_df.loc[music_df['id'] == '567438.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-12')
music_df.loc[music_df['id'] == '567438.html', 
             'date_by_hand_mics_etc'] = '.001'
#1498
music_df.loc[music_df['id'] == '567513.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '567513.html', 
             'date_by_hand_mics_etc'] = '_chatham_track_12'
#1523
music_df.loc[music_df['id'] == '567572.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-22')
music_df.loc[music_df['id'] == '567572.html', 
             'date_by_hand_mics_etc'] = '_chatham_track_18'
#1526
music_df.loc[music_df['id'] == '567578.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-08')
music_df.loc[music_df['id'] == '567578.html', 
             'date_by_hand_mics_etc'] = '-trembling bells aud01'
#1545
music_df.loc[music_df['id'] == '567639.html', 
             'date_by_hand'] = pandas.to_datetime('1975-01-03')
music_df.loc[music_df['id'] == '567639.html', 
             'date_by_hand_mics_etc'] = ' u t'
#1551
music_df.loc[music_df['id'] == '567649.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567649.html', 
             'date_by_hand_mics_etc'] = '_rochester_track_20'
#1554
music_df.loc[music_df['id'] == '567662.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567662.html', 
             'date_by_hand_mics_etc'] = '_rochesterr_track_04'
#1557
music_df.loc[music_df['id'] == '567671.html', 
             'date_by_hand'] = pandas.to_datetime('1989-11-19')
music_df.loc[music_df['id'] == '567671.html', 
             'date_by_hand_mics_etc'] = '_07'
#1598
music_df.loc[music_df['id'] == '567750.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-23')
music_df.loc[music_df['id'] == '567750.html', 
             'date_by_hand_mics_etc'] = '_rochesterr_track_07'
#1606
music_df.loc[music_df['id'] == '567790.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-26')
music_df.loc[music_df['id'] == '567790.html', 
             'date_by_hand_mics_etc'] = '.aud.mk4.petaluma.flac16 01'
#1634
music_df.loc[music_df['id'] == '567872.html', 
             'date_by_hand'] = pandas.to_datetime('2016-07-30')
music_df.loc[music_df['id'] == '567872.html', 
             'date_by_hand_mics_etc'] = '_07'
#1677
music_df.loc[music_df['id'] == '126880.html', 
             'date_by_hand'] = pandas.to_datetime('1988-10-01')

music_df['date_by_hand_mics_etc'] = music_df['date_by_hand_mics_etc'].fillna('')
#music_df[['id', 'filename']][
#    (music_df['date_by_hand'].isnull()) 
#    & (music_df['date_format'] == '')
#].loc[1757:]

music_df.loc[music_df['date_by_hand'].notnull(), 'date_format'] = 'by_hand'

music_df.loc[music_df['date_format'] == 'by_hand', 'parsed_date'] = music_df['date_by_hand']
music_df.loc[music_df['date_format'] == 'by_hand', 'mics_etc'] = music_df['date_by_hand_mics_etc']

music_df = music_df.drop(['date_by_hand', 'date_by_hand_mics_etc'], axis=1)

#music_df[['id', 'filename', 'date_format', 'parsed_date', 'mics_etc']][music_df['date_format'] == 'by_hand']
music_df.groupby('date_format').size()


date_format
              284 
by_hand       56  
date1         69  
date2         96  
date3         5   
yy-mm-dd      123 
yyyy-mm-dd    1170
dtype: int64

In [73]:
music_df['theyear'] = np.nan
music_df['themonth'] = np.nan
music_df['theyear_mics_etc'] = ''

#147
music_df.loc[music_df['id'] == 'e059e8d39da4b75145b1e434b0569df9c8f59367', 'theyear'] = 1995
#174
music_df.loc[music_df['id'] == '8a7be6c05e405264de580672fbe6619a79955c43', 'theyear'] = 1975
music_df.loc[music_df['id'] == '8a7be6c05e405264de580672fbe6619a79955c43', 'theyear_mics_etc'] = 'tt01'
#179
music_df.loc[music_df['id'] == 'jg1975-xx-xx.boswell-smith.flac16', 'theyear'] = 1975
music_df.loc[music_df['id'] == 'jg1975-xx-xx.boswell-smith.flac16', 'theyear_mics_etc'] = 'tt01'
#208
music_df.loc[music_df['id'] == 'c23154fca319e57ed53f15a6a84e87a539312555', 'theyear'] = 1968
music_df.loc[music_df['id'] == 'c23154fca319e57ed53f15a6a84e87a539312555', 'theyear_mics_etc'] = ' los angeles,ca jonathan winters show mk1-tv-01'
#209
music_df.loc[music_df['id'] == '17a8a1fb659a7ccbc3df9d710ca2b1be9504c5c4', 'theyear'] = 1968
music_df.loc[music_df['id'] == '17a8a1fb659a7ccbc3df9d710ca2b1be9504c5c4', 'theyear_mics_etc'] = ' los angeles,ca jonathan winters show mk2-tv-01'
#314
music_df.loc[music_df['id'] == '587452.html', 'theyear'] = 1975
music_df.loc[music_df['id'] == '587452.html', 'themonth'] = 3
music_df.loc[music_df['id'] == '587452.html', 'theyear_mics_etc'] = 't01'
#616
music_df.loc[music_df['id'] == '587815.html', 'theyear'] = 2016
music_df.loc[music_df['id'] == '587815.html', 'theyear_mics_etc'] = 't01'
#930
music_df.loc[music_df['id'] == '566034.html', 'theyear'] = 1978
music_df.loc[music_df['id'] == '566034.html', 'theyear_mics_etc'] = '-gettysburg-tr01'
#938
music_df.loc[music_df['id'] == '566072.html', 'theyear'] = 1985
music_df.loc[music_df['id'] == '566072.html', 'themonth'] = 10
music_df.loc[music_df['id'] == '566072.html', 'theyear_mics_etc'] = 'd01t01'
#956
music_df.loc[music_df['id'] == '566119.html', 'theyear'] = 2016
#990
music_df.loc[music_df['id'] == '566208.html', 'theyear'] = 1972
music_df.loc[music_df['id'] == '566208.html', 'themonth'] = 1
#1095
music_df.loc[music_df['id'] == '566506.html', 'theyear'] = 2016
music_df.loc[music_df['id'] == '566506.html', 'themonth'] = 7
music_df.loc[music_df['id'] == '566506.html', 'theyear_mics_etc'] = '-track01'
#1198
music_df.loc[music_df['id'] == '566758.html', 'theyear'] = 1991
music_df.loc[music_df['id'] == '566758.html', 'themonth'] = 7
music_df.loc[music_df['id'] == '566758.html', 'theyear_mics_etc'] = 't01'
#1223
music_df.loc[music_df['id'] == '566814.html', 'theyear'] = 2016
music_df.loc[music_df['id'] == '566814.html', 'themonth'] = 1
#1325
music_df.loc[music_df['id'] == '567117.html', 'theyear'] = 1986
music_df.loc[music_df['id'] == '567117.html', 'themonth'] = 6
music_df.loc[music_df['id'] == '567117.html', 'theyear_mics_etc'] = '-tigers-t01'
#1355
music_df.loc[music_df['id'] == '567186.html', 'theyear'] = 1986
music_df.loc[music_df['id'] == '567186.html', 'themonth'] = 6
music_df.loc[music_df['id'] == '567186.html', 'theyear_mics_etc'] = '-paradise-t01'
#1424
music_df.loc[music_df['id'] == '567323.html', 'theyear'] = 1983
music_df.loc[music_df['id'] == '567323.html', 'theyear_mics_etc'] = ' -cd2'
#1427
music_df.loc[music_df['id'] == '567333.html', 'theyear'] = 2007
music_df.loc[music_df['id'] == '567333.html', 'themonth'] = 7
music_df.loc[music_df['id'] == '567333.html', 'theyear_mics_etc'] = ' silver machine and waiting for the man'
#1448
music_df.loc[music_df['id'] == '567394.html', 'theyear'] = 1986
music_df.loc[music_df['id'] == '567394.html', 'themonth'] = 5
music_df.loc[music_df['id'] == '567394.html', 'theyear_mics_etc'] = '-fm(1st-gen)t04'
#1564
music_df.loc[music_df['id'] == '567683.html', 'theyear'] = 2017
music_df.loc[music_df['id'] == '567683.html', 'theyear_mics_etc'] = ' (seen the lights go out on broadway)'
#1566
music_df.loc[music_df['id'] == '567686.html', 'theyear'] = 1968
music_df.loc[music_df['id'] == '567686.html', 'theyear_mics_etc'] = ' los angeles,ca jonathan winters show mk1-tv-01'
#1567
music_df.loc[music_df['id'] == '567687.html', 'theyear'] = 1968
music_df.loc[music_df['id'] == '567687.html', 'theyear_mics_etc'] = ' los angeles,ca jonathan winters show mk2-tv-01'
#1612
music_df.loc[music_df['id'] == '567810.html', 'theyear'] = 2016
music_df.loc[music_df['id'] == '567810.html', 'theyear_mics_etc'] = ' - matthew shipp'
#1737
music_df.loc[music_df['id'] == '127033.html', 'theyear'] = 1991
music_df.loc[music_df['id'] == '127033.html', 'theyear_mics_etc'] = ' 44khz16b t01'
#1748
music_df.loc[music_df['id'] == '127062.html', 'theyear'] = 2017
music_df.loc[music_df['id'] == '127062.html', 'theyear_mics_etc'] = ' (seen the lights go out on broadway)'

music_df.loc[
    (music_df['date_format'] == '') &
    (music_df['theyear'].notnull()) & 
    (music_df['themonth'].notnull())
    , 'date_format'] = 'year_month_by_hand'

music_df.loc[
    (music_df['date_format'] == '') &
    (music_df['theyear'].notnull())
    , 'date_format'] = 'year_by_hand'

music_df.loc[music_df['theyear_mics_etc'] != '', 'mics_etc'] = music_df['theyear_mics_etc']

music_df = music_df.drop(['theyear_mics_etc'], axis=1)

music_df.loc[music_df['date_format'] == '', 'date_format'] = 'nodate'
music_df.groupby('date_format').size()



date_format
by_hand               56  
date1                 69  
date2                 96  
date3                 5   
nodate                259 
year_by_hand          15  
year_month_by_hand    10  
yy-mm-dd              123 
yyyy-mm-dd            1170
dtype: int64

In [74]:
music_df.sample(100)

,id,source,filename,filetype,remainder,band,date_right_after_band,mics_etc,abbrev_type,jeremy_band_finding_notes,parsed_date,date_format,after_first_digit,theyear,themonth
294,587427.html,Etree,bsb2016-06-24t01.flac,flac,bsb2016-06-24t01,bsb,True,,notband,,2016-06-24,yyyy-mm-dd,2016-06-24t01,NaN,NaN
293,587425.html,Etree,gd1987-07-04-24bits01t01.flac,flac,gd1987-07-04-24bits01t01,gd,True,,canonical,,1987-07-04,yyyy-mm-dd,1987-07-04-24bits01t01,NaN,NaN
132,6d8058dd7686c16e17e9c9f7bfb7d84439611051,LosslessLegs,gd93-01-24d1t01.shn,shn,gd93-01-24d1t01,gd,True,,canonical,,1993-01-24,yy-mm-dd,93-01-24d1t01,NaN,NaN
1305,567041.html,DimeaDozen,atomic rooster 2016-07-14 t01.flac,flac,atomic rooster 2016-07-14 t01,atomic rooster,False,,jeremy,,2016-07-14,yyyy-mm-dd,2016-07-14 t01,NaN,NaN
1158,566663.html,DimeaDozen,1988-04-10 - century hall - nagoya - 101 rhythm of love.flac,flac,1988-04-10 - century hall - nagoya - 101 rhythm of love,,True,,blank,,1988-04-10,yyyy-mm-dd,1988-04-10 - century hall - nagoya - 101 rhythm of love,NaN,NaN
1135,566610.html,DimeaDozen,jam_19820925_liverpoolmc201.flac,flac,jam_19820925_liverpoolmc201,jam,False,,jeremy,,1982-09-25,date2,19820925_liverpoolmc201,NaN,NaN
252,587380.html,Etree,gm2001-04-05d1t01.flac,flac,gm2001-04-05d1t01,gm,True,,variant,,2001-04-05,yyyy-mm-dd,2001-04-05d1t01,NaN,NaN
1046,566360.html,DimeaDozen,ravi_shankar-1998-02-03-tokyo-tr01-tuning.flac,flac,ravi_shankar-1998-02-03-tokyo-tr01-tuning,ravi_shankar,False,,jeremy,,1998-02-03,yyyy-mm-dd,1998-02-03-tokyo-tr01-tuning,NaN,NaN
1133,566608.html,DimeaDozen,avett brothers 2016-03-06 okeechobee 01 satan pulls the strings.flac,flac,avett brothers 2016-03-06 okeechobee 01 satan pulls the strings,avett brothers,False,,jeremy,,2016-03-06,yyyy-mm-dd,2016-03-06 okeechobee 01 satan pulls the strings,NaN,NaN
1427,567333.html,DimeaDozen,07-07 silver machine and waiting for the man.flac,flac,07-07 silver machine and waiting for the man,,True,silver machine and waiting for the man,blank,,NaT,year_month_by_hand,07-07 silver machine and waiting for the man,2007.0,7.0
